In [1]:
import numpy as np
import pandas as pd
import logging
import concurrent.futures
from multiprocessing.dummy import Pool as ThreadPool
import time
import requests 
from datetime import datetime

In [2]:
#Fetch api response
def fetchStockData(stockName):
    try:
        #Build api parameters
        params = dict()
        params["region"] = "CA"
        params["lang"] = "en-CA"
        params["includePrePost"] = "false"
        params["interval"]="1d"
        params["useYfid"] = "true"
        params["range"] ="6mo"
        params["corsDomain"] = "ca.finance.yahoo.com"
        params[".tsrc"] ="finance"

        headers = dict()
        headers["User-Agent"]= "Mozilla/5.0"

        webUrl = "https://query1.finance.yahoo.com/v8/finance/chart/{stockName}.TO".format(stockName =stockName)

        #start api request
        response=requests.get(webUrl, params = params, headers= headers)
        return response
    except Exception as e:
            print(e)
    return NULL

In [3]:
def fetchStockDataFromYahooFinanceWebApi(stockName):
        try:
            response = fetchStockData(stockName)
            if response.ok == True:
                chart = response.json()["chart"]
                result = chart["result"]
                firstItem = result[0]
                timestamps = firstItem["timestamp"]
                indicators = firstItem["indicators"]["quote"][0]
                
                dataFrame = pd.DataFrame(indicators)
                dataFrame["time"] = timestamps
                dataFrame["stock"] = stockName
                
                dataFrame["date"] = np.array(len(timestamps))
                
                stockDatas = list()
                for index, value in enumerate(timestamps):
                    dataFrame["date"][index] = datetime.fromtimestamp(value)
                
                return dataFrame
        except Exception as e:
            print(e)
        return None

In [4]:
def readStockNames():
    tickers=pd.read_csv('../data_storage/stock_ticker.csv')
    return tickers

In [5]:
stocks = readStockNames()["Ticker_Symbol_tsx"]
stocks

0     BB
1    PKI
2    DII
3    DML
4    ONC
5    EQB
Name: Ticker_Symbol_tsx, dtype: object

In [6]:
def storeFinanceDataForStock(stocks):
    dataFrame = pd.DataFrame()
    for i in stocks:
        dataFrame = dataFrame.append(fetchStockDataFromYahooFinanceWebApi(i))
        dataFrame.to_csv('../data_storage/yahoo_finance_stocks_data.csv')

In [7]:
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as p:
    p.map(storeFinanceDataForStock(stocks))

/var/folders/x0/qxhk7kb567vdwjjvpqvxdjv00000gn/T/ipykernel_23915/1601328541.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame["date"][index] = datetime.fromtimestamp(value)
/var/folders/x0/qxhk7kb567vdwjjvpqvxdjv00000gn/T/ipykernel_23915/943335011.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFrame = dataFrame.append(fetchStockDataFromYahooFinanceWebApi(i))
/var/folders/x0/qxhk7kb567vdwjjvpqvxdjv00000gn/T/ipykernel_23915/1601328541.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame["date"][index] 

In [8]:
# Verify the dataset
pd.read_csv('../data_storage/yahoo_finance_stocks_data.csv')

,Unnamed: 0,low,volume,close,open,high,time,stock,date
0,0,7.540000,1275300,7.690000,7.650000,7.740000,1654695000,BB,2022-06-08 09:30:00
1,1,7.350000,1223400,7.350000,7.680000,7.690000,1654781400,BB,2022-06-09 09:30:00
2,2,7.020000,1518100,7.080000,7.190000,7.320000,1654867800,BB,2022-06-10 09:30:00
3,3,6.600000,1456700,6.610000,6.790000,6.890000,1655127000,BB,2022-06-13 09:30:00
4,4,6.540000,1654300,6.680000,6.670000,6.810000,1655213400,BB,2022-06-14 09:30:00
...,...,...,...,...,...,...,...,...,...
635,123,59.590000,52000,60.099998,59.799999,60.740002,1669991400,EQB,2022-12-02 09:30:00
636,124,58.500000,57300,59.189999,60.130001,60.299999,1670250600,EQB,2022-12-05 09:30:00
637,125,59.000000,61600,59.700001,59.160000,60.119999,1670337000,EQB,2022-12-06 09:30:00
638,126,59.389999,51600,60.200001,59.700001,60.570000,1670423400,EQB,2022-12-07 09:30:00
